In [64]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from sklearn.model_selection import train_test_split
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [65]:
import glob
files = glob.glob("../../datasets/xerces-1.4.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [66]:
ant_master

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,xerces,1.4.4,org.apache.xerces.impl.xs.traversers.XSAttribu...,12,1,0,35,75,14,16,...,0.559140,19,0.000000,0.209790,0,0,341.416667,31,6.6667,13
1,xerces,1.4.4,org.apache.xerces.impl.dv.xs.DateTimeDatatypeV...,3,4,0,6,12,3,2,...,0.000000,0,0.985714,0.428571,1,7,21.666667,1,0.3333,1
2,xerces,1.4.4,org.apache.html.dom.HTMLOptGroupElementImpl,5,4,0,2,11,10,0,...,0.000000,0,0.959184,0.450000,0,0,5.000000,1,0.8000,2
3,xerces,1.4.4,org.apache.wml.WMLPElement,6,1,0,2,6,15,1,...,0.000000,0,0.000000,0.750000,0,0,0.000000,1,1.0000,1
4,xerces,1.4.4,org.w3c.dom.html.HTMLTitleElement,2,1,0,0,2,1,0,...,0.000000,0,0.000000,0.750000,0,0,0.000000,1,1.0000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,xerces,1.4.4,org.apache.wml.WMLRefreshElement,0,1,0,2,0,0,1,...,0.000000,0,0.000000,0.000000,0,0,0.000000,0,0.0000,1
584,xerces,1.4.4,org.apache.xerces.impl.dv.dtd.ListDatatypeVali...,12,2,0,9,39,20,2,...,0.888889,1,0.230769,0.273810,0,0,45.583333,2,1.0000,2
585,xerces,1.4.4,org.apache.xerces.impl.xs.models.XSSimpleCM,6,1,0,7,12,0,1,...,1.000000,2,0.000000,0.416667,0,0,39.000000,13,3.5000,1
586,xerces,1.4.4,org.apache.xerces.impl.dtd.models.CMStateSet,12,1,0,10,18,0,10,...,0.000000,0,0.000000,0.416667,1,1,41.666667,7,3.7500,2


In [67]:
train, test = train_test_split(ant_master, test_size=0.2)

In [68]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [69]:
x_train

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
28,8,1,0,1,14,0,1,0,5,0.142857,574,0.0,0,0.000000,0.520833,0,0,70.375000,8,5.6250
386,14,1,0,0,14,91,0,0,14,2.000000,14,0.0,0,0.000000,0.206349,0,0,0.000000,1,1.0000
149,8,1,0,0,8,28,0,0,8,2.000000,8,0.0,0,0.000000,0.354167,0,0,0.000000,1,1.0000
585,6,1,0,7,12,0,1,6,6,0.766667,246,1.0,2,0.000000,0.416667,0,0,39.000000,13,3.5000
444,4,3,0,4,12,0,1,4,1,0.666667,58,0.5,0,0.952381,0.500000,2,7,13.000000,2,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,8,1,0,0,8,28,0,0,8,2.000000,8,0.0,0,0.000000,0.750000,0,0,0.000000,1,1.0000
196,3,1,0,5,10,0,3,2,3,0.166667,41,0.0,2,0.000000,0.500000,0,0,11.666667,1,0.6667
121,3,1,0,6,3,3,3,3,3,2.000000,3,0.0,0,0.000000,0.666667,0,0,0.000000,1,1.0000
412,13,4,0,3,18,78,0,3,13,2.000000,73,0.0,0,0.872340,0.403846,1,6,4.615385,1,0.9231


In [70]:
x_test

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
245,3,1,2,5,6,1,2,3,1,1.166667,35,1.0,1,0.000000,0.555556,0,0,9.666667,1,0.6667
525,18,1,2,14,68,0,2,12,7,0.882353,1804,1.0,8,0.000000,0.227273,0,0,97.722222,3,1.0556
314,12,1,0,0,12,66,0,0,12,2.000000,12,0.0,0,0.000000,0.500000,0,0,0.000000,1,1.0000
469,26,1,0,0,26,325,0,0,26,2.000000,26,0.0,0,0.000000,0.500000,0,0,0.000000,1,1.0000
514,4,4,2,48,6,0,48,0,4,0.333333,27,1.0,0,0.944444,0.666667,0,0,5.500000,1,0.2500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,2,1,0,0,2,1,0,0,2,2.000000,2,0.0,0,0.000000,0.750000,0,0,0.000000,1,1.0000
310,26,1,0,0,40,0,0,0,24,0.080000,763,0.0,0,0.000000,0.528846,0,0,28.269231,5,2.6538
86,11,1,0,5,11,55,2,3,11,1.100000,16,0.0,0,0.000000,0.327273,0,0,0.000000,1,1.0000
35,11,1,0,5,37,0,0,5,11,0.716667,332,1.0,3,0.000000,0.318182,0,0,28.636364,1,0.9091


In [71]:
y_train = train['bug']
y_test = test['bug']

In [72]:
y_train

28     1
386    0
149    0
585    1
444    2
      ..
380    0
196    1
121    3
412    2
20     1
Name: bug, Length: 470, dtype: int64

In [73]:
y_test

245     1
525    21
314     0
469     0
514     2
       ..
462     0
310     1
86      1
35      2
578     0
Name: bug, Length: 118, dtype: int64

In [74]:
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)

In [75]:
x_train_np

array([[ 8.        ,  1.        ,  0.        , ..., 70.375     ,
         8.        ,  5.625     ],
       [14.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ],
       [ 8.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ],
       ...,
       [ 3.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ],
       [13.        ,  4.        ,  0.        , ...,  4.61538461,
         1.        ,  0.9231    ],
       [10.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ]])

In [76]:
x_test_np

array([[ 3.        ,  1.        ,  2.        , ...,  9.66666667,
         1.        ,  0.6667    ],
       [18.        ,  1.        ,  2.        , ..., 97.72222222,
         3.        ,  1.0556    ],
       [12.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ],
       ...,
       [11.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ],
       [11.        ,  1.        ,  0.        , ..., 28.63636364,
         1.        ,  0.9091    ],
       [ 2.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ]])

In [77]:
train_x = np.reshape(x_train_np, (x_train_np.shape[0], 1, x_train_np.shape[1]))
test_x = np.reshape(x_test_np, (x_test_np.shape[0], 1, x_test_np.shape[1]))

In [78]:
train_y = y_train.to_numpy()
test_y = y_test.to_numpy()

In [79]:
train_x

array([[[ 8.        ,  1.        ,  0.        , ..., 70.375     ,
          8.        ,  5.625     ]],

       [[14.        ,  1.        ,  0.        , ...,  0.        ,
          1.        ,  1.        ]],

       [[ 8.        ,  1.        ,  0.        , ...,  0.        ,
          1.        ,  1.        ]],

       ...,

       [[ 3.        ,  1.        ,  0.        , ...,  0.        ,
          1.        ,  1.        ]],

       [[13.        ,  4.        ,  0.        , ...,  4.61538461,
          1.        ,  0.9231    ]],

       [[10.        ,  1.        ,  0.        , ...,  0.        ,
          1.        ,  1.        ]]])

In [80]:
test_y

array([ 1, 21,  0,  0,  2,  1,  7,  0,  3, 15,  0,  1,  1,  0,  2,  0,  1,
        2,  1,  1,  0,  1,  1,  0, 15,  2,  2,  0,  1,  0,  2,  2,  2,  2,
        0,  5,  1, 15,  1,  1,  0,  1,  1,  2, 11,  1,  0,  0,  0,  1,  2,
        0,  0,  2,  5,  0,  1,  4,  2,  0,  1,  1,  1,  2,  2,  6,  7,  0,
        1,  4,  0,  0,  1, 62,  6,  0,  2,  1,  0, 18,  2,  0,  8,  0,  1,
        0,  2,  2,  1,  1,  1,  1,  1,  2,  9,  1,  6,  0,  0,  1,  3, 17,
        1,  1,  1,  2,  1,  8,  4,  1,  4,  2,  2,  0,  1,  1,  2,  0])

In [81]:
# Designing and initializing the model.
from keras.layers import LSTM, Dense, SimpleRNN
# Designing and initializing the model.
model = Sequential()
model.add(LSTM(200, return_sequences=True))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(50, dropout = 0.2, return_sequences=True))
model.add(LSTM(20, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.0001) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [82]:
# Fitting the model on training data.
import datetime
log_dir = '/home/bavanya/Desktop/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_x, train_y, epochs = 100, callbacks=[tensorboard_callback])

Epoch 1/100
15/15 [==============================] - 5s 25ms/step - loss: 29.6851 - mse: 29.6851 - mae: 2.6465 - root_mean_squared_error: 5.4484 - mean_squared_logarithmic_error: 1.4119
Epoch 2/100
15/15 [==============================] - 0s 6ms/step - loss: 29.5981 - mse: 29.5981 - mae: 2.6381 - root_mean_squared_error: 5.4404 - mean_squared_logarithmic_error: 1.3842
Epoch 3/100
15/15 [==============================] - 0s 6ms/step - loss: 29.4897 - mse: 29.4897 - mae: 2.6274 - root_mean_squared_error: 5.4304 - mean_squared_logarithmic_error: 1.3505
Epoch 4/100
15/15 [==============================] - 0s 6ms/step - loss: 29.3520 - mse: 29.3520 - mae: 2.6136 - root_mean_squared_error: 5.4178 - mean_squared_logarithmic_error: 1.3097
Epoch 5/100
15/15 [==============================] - 0s 6ms/step - loss: 29.1441 - mse: 29.1441 - mae: 2.5921 - root_mean_squared_error: 5.3985 - mean_squared_logarithmic_error: 1.2505
Epoch 6/100
15/15 [==============================] - 0s 6ms/step - loss: 2

15/15 [==============================] - 0s 6ms/step - loss: 17.9189 - mse: 17.9189 - mae: 1.8341 - root_mean_squared_error: 4.2331 - mean_squared_logarithmic_error: 0.3246
Epoch 45/100
15/15 [==============================] - 0s 5ms/step - loss: 17.8380 - mse: 17.8380 - mae: 1.8238 - root_mean_squared_error: 4.2235 - mean_squared_logarithmic_error: 0.3211
Epoch 46/100
15/15 [==============================] - 0s 5ms/step - loss: 17.8117 - mse: 17.8117 - mae: 1.8233 - root_mean_squared_error: 4.2204 - mean_squared_logarithmic_error: 0.3231
Epoch 47/100
15/15 [==============================] - 0s 6ms/step - loss: 17.7671 - mse: 17.7671 - mae: 1.8110 - root_mean_squared_error: 4.2151 - mean_squared_logarithmic_error: 0.3176
Epoch 48/100
15/15 [==============================] - 0s 5ms/step - loss: 17.6717 - mse: 17.6717 - mae: 1.8124 - root_mean_squared_error: 4.2038 - mean_squared_logarithmic_error: 0.3183
Epoch 49/100
15/15 [==============================] - 0s 5ms/step - loss: 17.6152 -

15/15 [==============================] - 0s 7ms/step - loss: 15.2982 - mse: 15.2982 - mae: 1.5206 - root_mean_squared_error: 3.9113 - mean_squared_logarithmic_error: 0.2247
Epoch 89/100
15/15 [==============================] - 0s 6ms/step - loss: 15.2268 - mse: 15.2268 - mae: 1.5328 - root_mean_squared_error: 3.9022 - mean_squared_logarithmic_error: 0.2241
Epoch 90/100
15/15 [==============================] - 0s 5ms/step - loss: 15.1520 - mse: 15.1520 - mae: 1.5249 - root_mean_squared_error: 3.8926 - mean_squared_logarithmic_error: 0.2247
Epoch 91/100
15/15 [==============================] - 0s 5ms/step - loss: 15.0464 - mse: 15.0464 - mae: 1.4995 - root_mean_squared_error: 3.8790 - mean_squared_logarithmic_error: 0.2161
Epoch 92/100
15/15 [==============================] - 0s 7ms/step - loss: 15.0106 - mse: 15.0106 - mae: 1.4906 - root_mean_squared_error: 3.8743 - mean_squared_logarithmic_error: 0.2107
Epoch 93/100
15/15 [==============================] - 0s 6ms/step - loss: 14.9952 -

In [83]:
%tensorboard --logdir '/home/bavanya/Desktop/tensorboard/'

Reusing TensorBoard on port 6006 (pid 3750), started 3:12:27 ago. (Use '!kill 3750' to kill it.)

In [84]:
score = model.evaluate(test_x, test_y)
score

4/4 [==============================] - 1s 2ms/step - loss: 36.6826 - mse: 36.6826 - mae: 2.1418 - root_mean_squared_error: 6.0566 - mean_squared_logarithmic_error: 0.3215


[36.682552337646484,
 36.682552337646484,
 2.1418113708496094,
 6.056612491607666,
 0.321526437997818]